In [1]:
import numpy as np
import pandas as pd
import os

def calculate_log_returns(portfolio_values):
    """
    从投资组合价值序列计算对数收益率。
    """
    if isinstance(portfolio_values, pd.Series):
        values_arr = portfolio_values.to_numpy()
    else:
        values_arr = np.array(portfolio_values)

    if len(values_arr) < 2:
        raise ValueError("投资组合价值序列必须至少有两个数据点才能计算收益率。")
    if np.any(values_arr <= 0):
        raise ValueError("投资组合价值必须全部为正数才能计算对数收益率。")
        
    log_returns = np.log(values_arr[1:] / values_arr[:-1])
    return log_returns

def calculate_annualized_sharpe_ratio(returns, risk_free_rate=0, periods_per_year=252):
    """
    从收益率序列计算年化夏普比率。
    """
    if len(returns) < 2:
        return np.nan
    risk_free_rate_per_period = risk_free_rate / periods_per_year
    excess_returns = returns - risk_free_rate_per_period
    mean_excess_return = np.mean(excess_returns)
    std_dev_excess_return = np.std(excess_returns)
    
    if std_dev_excess_return == 0:
        return np.inf if mean_excess_return > 1e-9 else (0 if abs(mean_excess_return) < 1e-9 else -np.inf)
            
    sharpe_ratio_annualized = (mean_excess_return / std_dev_excess_return) * np.sqrt(periods_per_year)
    return sharpe_ratio_annualized

def circular_block_bootstrap_comparison(
    drl_portfolio_values, 
    baseline_portfolio_values,
    risk_free_rate=0,
    periods_per_year=252,
    num_bootstrap_samples=10000,
    bootstrap_seed=None,
    block_size=22
):
    """
    执行循环块自举法 (Circular Block Bootstrap) 来对比DRL和基准模型的夏普比率。
    此方法将数据序列视为环形，解决了移动块自举法的边界效应问题。
    """
    if len(drl_portfolio_values) != len(baseline_portfolio_values):
        raise ValueError("DRL和基准模型的价值序列长度必须相同。")
    
    # 计算对数收益率序列
    drl_returns = calculate_log_returns(drl_portfolio_values)
    baseline_returns = calculate_log_returns(baseline_portfolio_values)

    # 计算原始的夏普比率和它们的差值
    drl_sharpe_observed = calculate_annualized_sharpe_ratio(drl_returns, risk_free_rate, periods_per_year)
    baseline_sharpe_observed = calculate_annualized_sharpe_ratio(baseline_returns, risk_free_rate, periods_per_year)
    observed_difference = drl_sharpe_observed - baseline_sharpe_observed

    # 设置随机种子以保证可复现性
    if bootstrap_seed is not None:
        np.random.seed(bootstrap_seed)

    num_returns = len(drl_returns)
    
    if num_returns < block_size:
        raise ValueError(f"数据长度({num_returns})不能小于块大小({block_size})。")

    bootstrap_differences = []

    for _ in range(num_bootstrap_samples):
        # 循环块自举法的核心逻辑
        num_blocks_to_sample = int(np.ceil(num_returns / block_size))
        
        # 在循环法中，任何一个点都可以作为起始点
        block_start_indices = np.random.choice(num_returns, size=num_blocks_to_sample, replace=True)

        bootstrap_drl_returns_list = []
        bootstrap_baseline_returns_list = []
        
        for start_idx in block_start_indices:
            # 生成带有“环绕”效果的区块索引
            # 例如，在长度为100的序列中，从95开始取10个，索引将是95,96,97,98,99,0,1,2,3,4
            block_indices = (start_idx + np.arange(block_size)) % num_returns
            
            # 使用生成的环形索引来提取数据块
            drl_block = drl_returns[block_indices]
            baseline_block = baseline_returns[block_indices]
            
            bootstrap_drl_returns_list.append(drl_block)
            bootstrap_baseline_returns_list.append(baseline_block)
        
        # 将抽取的区块拼接成一个新的时间序列，并截取到原始长度
        bootstrap_drl_returns = np.concatenate(bootstrap_drl_returns_list)[:num_returns]
        bootstrap_baseline_returns = np.concatenate(bootstrap_baseline_returns_list)[:num_returns]

        # 计算自举样本的夏普比率
        bootstrap_drl_sr = calculate_annualized_sharpe_ratio(bootstrap_drl_returns, risk_free_rate, periods_per_year)
        bootstrap_baseline_sr = calculate_annualized_sharpe_ratio(bootstrap_baseline_returns, risk_free_rate, periods_per_year)
        
        # 仅在夏普比率有效时记录差值
        if not (np.isnan(bootstrap_drl_sr) or np.isinf(bootstrap_drl_sr) or \
                np.isnan(bootstrap_baseline_sr) or np.isinf(bootstrap_baseline_sr)):
            bootstrap_differences.append(bootstrap_drl_sr - bootstrap_baseline_sr)
    
    bootstrap_differences = np.array(bootstrap_differences)
    
    # 计算 p 值 (H_A: SR_DRL > SR_Baseline)
    p_value_drl_greater_baseline = np.mean(bootstrap_differences <= 0)

    # 计算 95% 置信区间
    lower_bound = np.percentile(bootstrap_differences, 2.5)
    upper_bound = np.percentile(bootstrap_differences, 97.5)
    confidence_interval_difference_95 = (lower_bound, upper_bound)

    return {
        'drl_sharpe_observed': drl_sharpe_observed,
        'baseline_sharpe_observed': baseline_sharpe_observed,
        'observed_difference_SR': observed_difference,
        'p_value_drl_greater_baseline': p_value_drl_greater_baseline,
        'confidence_interval_difference_95_SR': confidence_interval_difference_95,
        'bootstrap_SR_differences_count': len(bootstrap_differences),
        'method': f'Circular Block Bootstrap (block size={block_size})' # 更新方法名称
    }

def print_results_table(results_df):
    """
    以表格形式打印所有显著性检验结果
    """
    print("\n" + "="*140)
    print("DRL模型 vs 基准模型显著性检验结果汇总")
    print("="*140)
    
    # 表头
    print(f"{'DRL模型':<15} {'基准模型':<15} {'DRL夏普比率':<12} {'基准夏普比率':<12} {'差值':<10} {'P值':<8} {'95%置信区间':<25} {'显著性':<10}")
    print("-"*140)
    
    # 每行结果
    for _, row in results_df.iterrows():
        if not pd.isna(row['P_Value']):
            significance = "***" if row['P_Value'] < 0.01 else "**" if row['P_Value'] < 0.05 else "*" if row['P_Value'] < 0.1 else ""
            
            print(f"{row['DRL_Model']:<15} {row['Baseline_Model']:<15} {row['DRL_Sharpe']:<12.4f} "
                  f"{row['Baseline_Sharpe']:<12.4f} {row['Difference']:<10.4f} {row['P_Value']:<8.4f} "
                  f"({row['CI_Lower']:.4f}, {row['CI_Upper']:.4f}){'':<8} {significance:<10}")
        else:
            print(f"{row['DRL_Model']:<15} {row['Baseline_Model']:<15} {'错误':<12} {'--':<12} {'--':<10} {'--':<8} {'--':<25} {'--':<10}")
    
    print("-"*140)
    print("显著性标记: *** p<0.01, ** p<0.05, * p<0.1")
    print("="*140)

def save_results_to_csv(results_df, filename):
    """
    保存结果到CSV文件
    """
    # 添加显著性标记列
    def get_significance(p_val):
        if pd.isna(p_val):
            return "Error"
        elif p_val < 0.01:
            return "***"
        elif p_val < 0.05:
            return "**"
        elif p_val < 0.1:
            return "*"
        else:
            return "Not Significant"
    
    results_df['Significance'] = results_df['P_Value'].apply(get_significance)
    
    # 保存到CSV
    results_df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n结果已保存到: {filename}")

def main():
    # DRL模型文件名列表
    drl_models = [
        'df_PPO_vgg1_Tanh_maxSharpe_arbitrage_seed1.csv',
        'df_PPO_vgg1_Tanh_maxSharpe_arbitrage_seed2.csv',
        'df_PPO_vgg1_Tanh_maxSharpe_arbitrage_seed3.csv',
        'df_PPO_vgg1_Tanh_maxSharpe_arbitrage_seed4.csv',
        'df_PPO_vgg1_Tanh_maxSharpe_arbitrage_seed5.csv',
        'df_PPO_vgg1_Tanh_maxSharpe_arbitrage_seed6.csv'
    ]
    
    # 所有基准模型文件名列表
    baseline_models = [
        'Values_MV_MinRisk.csv',
        'Values_MV_Sharpe.csv',
        'Values_CVaR_MinRisk.csv',
        'Values_CVaR_Sharpe.csv',
        'Values_EVaR_MinRisk.csv',
        'Values_EVaR_Sharpe.csv',
        'Values_RP_MinRisk.csv',
        'Values_RP_Sharpe.csv',
        'Values_HRP.csv',
        'Values_HERC.csv',
        'Values_NCO_MinRisk.csv'
    ]
    
    # 目录设置
    drl_dir = 'SP500_results'
    baseline_dir = 'SP500_other_OPT_results'
    output_file = 'SP500_significance_circular.csv' # 建议修改输出文件名以作区分
    
    # 关键参数设置
    BLOCK_SIZE = 22
    NUM_SAMPLES = 10000
    SEED = 123
    
    # 存储所有结果
    all_results = []
    
    print(f"开始对比 {len(drl_models)} 个DRL模型 与 {len(baseline_models)} 个基准模型")
    print(f"总计 {len(drl_models) * len(baseline_models)} 次显著性检验")
    # 更新打印信息
    print(f"检验方法: Circular Block Bootstrap (block size={BLOCK_SIZE}, samples={NUM_SAMPLES})")
    
    comparison_count = 0
    total_comparisons = len(drl_models) * len(baseline_models)
    
    # 对每个DRL模型和每个基准模型进行检验
    for drl_idx, drl_file in enumerate(drl_models):
        drl_path = os.path.join(drl_dir, drl_file)
        drl_name = drl_file.replace('df_PPO_vgg1_Tanh_maxSharpe_arbitrage_', '').replace('.csv', '')
        
        try:
            # 加载DRL模型数据
            drl_df = pd.read_csv(drl_path, index_col=0, parse_dates=True)
            drl_values = drl_df.iloc[:, 0]
            
            for baseline_idx, baseline_file in enumerate(baseline_models):
                comparison_count += 1
                baseline_path = os.path.join(baseline_dir, baseline_file)
                baseline_name = baseline_file.replace('Values_', '').replace('.csv', '')
                
                print(f"[{comparison_count}/{total_comparisons}] {drl_name} vs {baseline_name}")
                
                try:
                    # 加载基准模型数据
                    baseline_df = pd.read_csv(baseline_path, index_col=0, parse_dates=True)
                    baseline_values = baseline_df.iloc[:, 0]
                    
                    # 处理数据长度不匹配
                    if len(baseline_values) != len(drl_values):
                        min_length = min(len(drl_values), len(baseline_values))
                        drl_values_adj = drl_values.iloc[:min_length]
                        baseline_values_adj = baseline_values.iloc[:min_length]
                    else:
                        drl_values_adj = drl_values
                        baseline_values_adj = baseline_values
                    
                    # 执行显著性检验 (调用修改后的函数)
                    result = circular_block_bootstrap_comparison(
                        drl_portfolio_values=drl_values_adj,
                        baseline_portfolio_values=baseline_values_adj,
                        risk_free_rate=0,
                        periods_per_year=252,
                        num_bootstrap_samples=NUM_SAMPLES,
                        bootstrap_seed=SEED + comparison_count,
                        block_size=BLOCK_SIZE
                    )
                    
                    # 添加结果到列表
                    all_results.append({
                        'DRL_Model': drl_name,
                        'Baseline_Model': baseline_name,
                        'DRL_Sharpe': result['drl_sharpe_observed'],
                        'Baseline_Sharpe': result['baseline_sharpe_observed'],
                        'Difference': result['observed_difference_SR'],
                        'P_Value': result['p_value_drl_greater_baseline'],
                        'CI_Lower': result['confidence_interval_difference_95_SR'][0],
                        'CI_Upper': result['confidence_interval_difference_95_SR'][1],
                        'Bootstrap_Samples': result['bootstrap_SR_differences_count'],
                        'Method': result['method']
                    })
                    
                except Exception as e:
                    print(f"  ✗ 基准模型处理错误: {e}")
                    all_results.append({
                        'DRL_Model': drl_name,
                        'Baseline_Model': baseline_name,
                        'DRL_Sharpe': np.nan,
                        'Baseline_Sharpe': np.nan,
                        'Difference': np.nan,
                        'P_Value': np.nan,
                        'CI_Lower': np.nan,
                        'CI_Upper': np.nan,
                        'Bootstrap_Samples': np.nan,
                        'Method': 'Error'
                    })
        
        except Exception as e:
            print(f"✗ DRL模型 {drl_name} 加载失败: {e}")
            # 为该DRL模型的所有基准模型添加错误记录
            for baseline_file in baseline_models:
                baseline_name = baseline_file.replace('Values_', '').replace('.csv', '')
                all_results.append({
                    'DRL_Model': drl_name,
                    'Baseline_Model': baseline_name,
                    'DRL_Sharpe': np.nan,
                    'Baseline_Sharpe': np.nan,
                    'Difference': np.nan,
                    'P_Value': np.nan,
                    'CI_Lower': np.nan,
                    'CI_Upper': np.nan,
                    'Bootstrap_Samples': np.nan,
                    'Method': 'DRL_Error'
                })
    
    # 转换为DataFrame
    results_df = pd.DataFrame(all_results)
    
    # 打印结果表格
    print_results_table(results_df)
    
    # 保存结果到CSV
    save_results_to_csv(results_df, output_file)
    
    # 统计汇总
    successful_tests = len(results_df[~pd.isna(results_df['P_Value'])])
    significant_at_05 = len(results_df[results_df['P_Value'] < 0.05])
    significant_at_01 = len(results_df[results_df['P_Value'] < 0.01])
    
    print(f"\n统计汇总:")
    print(f"- 总比较次数: {total_comparisons}")
    print(f"- 成功完成检验的比较次数: {successful_tests}")
    print(f"- 在1%显著性水平下显著的比较次数: {significant_at_01}")
    print(f"- 在5%显著性水平下显著的比较次数: {significant_at_05}")
    
    # 按DRL模型汇总
    print(f"\n按DRL模型汇总 (在5%水平下显著优于基准的次数):")
    for drl_name in results_df['DRL_Model'].unique():
        if not drl_name: continue
        drl_results = results_df[results_df['DRL_Model'] == drl_name]
        significant_count = len(drl_results[drl_results['P_Value'] < 0.05])
        total_count = len(drl_results[~pd.isna(drl_results['P_Value'])])
        if total_count > 0:
            print(f"- {drl_name}: {significant_count}/{total_count}")

if __name__ == '__main__':
    main()


开始对比 6 个DRL模型 与 11 个基准模型
总计 66 次显著性检验
检验方法: Circular Block Bootstrap (block size=22, samples=10000)
[1/66] seed1 vs MV_MinRisk
[2/66] seed1 vs MV_Sharpe
[3/66] seed1 vs CVaR_MinRisk
[4/66] seed1 vs CVaR_Sharpe
[5/66] seed1 vs EVaR_MinRisk
[6/66] seed1 vs EVaR_Sharpe
[7/66] seed1 vs RP_MinRisk
[8/66] seed1 vs RP_Sharpe
[9/66] seed1 vs HRP
[10/66] seed1 vs HERC
[11/66] seed1 vs NCO_MinRisk
[12/66] seed2 vs MV_MinRisk
[13/66] seed2 vs MV_Sharpe
[14/66] seed2 vs CVaR_MinRisk
[15/66] seed2 vs CVaR_Sharpe
[16/66] seed2 vs EVaR_MinRisk
[17/66] seed2 vs EVaR_Sharpe
[18/66] seed2 vs RP_MinRisk
[19/66] seed2 vs RP_Sharpe
[20/66] seed2 vs HRP
[21/66] seed2 vs HERC
[22/66] seed2 vs NCO_MinRisk
[23/66] seed3 vs MV_MinRisk
[24/66] seed3 vs MV_Sharpe
[25/66] seed3 vs CVaR_MinRisk
[26/66] seed3 vs CVaR_Sharpe
[27/66] seed3 vs EVaR_MinRisk
[28/66] seed3 vs EVaR_Sharpe
[29/66] seed3 vs RP_MinRisk
[30/66] seed3 vs RP_Sharpe
[31/66] seed3 vs HRP
[32/66] seed3 vs HERC
[33/66] seed3 vs NCO_MinRisk
[34/66] s